In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [4]:
data_ball = pd.read_csv('training_dataset/DataSet/IPL_Ball_by_Ball_2008_2022.csv')
data_match = pd.read_csv('training_dataset/DataSet/IPL_Matches_Result_2008_2022.csv')
data_ipl_2k23_player_team=pd.read_csv('training_dataset/DataSet/Ipl_2023 _cricketers - Players.csv')

In [5]:
data_ball.head()

ID  innings  overs  ballnumber       batter          bowler  \
0  1312200        1      0           1  YBK Jaiswal  Mohammed Shami   
1  1312200        1      0           2  YBK Jaiswal  Mohammed Shami   
2  1312200        1      0           3   JC Buttler  Mohammed Shami   
3  1312200        1      0           4  YBK Jaiswal  Mohammed Shami   
4  1312200        1      0           5  YBK Jaiswal  Mohammed Shami   

   non-striker extra_type  batsman_run  extras_run  total_run  non_boundary  \
0   JC Buttler        NaN            0           0          0             0   
1   JC Buttler    legbyes            0           1          1             0   
2  YBK Jaiswal        NaN            1           0          1             0   
3   JC Buttler        NaN            0           0          0             0   
4   JC Buttler        NaN            0           0          0             0   

   isWicketDelivery player_out kind fielders_involved       BattingTeam  
0                 0        NaN  NaN               NaN  Rajasthan Royals  
1                 0        NaN  NaN               NaN  Rajasthan Royals  
2                 0        NaN  NaN               NaN  Rajasthan Royals  
3                 0        NaN  NaN               NaN  Rajasthan Royals  
4                 0        NaN  NaN               NaN  Rajasthan Royals

In [6]:
data_match.head()

ID       City        Date Season  MatchNumber  \
0  1312200  Ahmedabad  2022-05-29   2022        Final   
1  1312199  Ahmedabad  2022-05-27   2022  Qualifier 2   
2  1312198    Kolkata  2022-05-25   2022   Eliminator   
3  1312197    Kolkata  2022-05-24   2022  Qualifier 1   
4  1304116     Mumbai  2022-05-22   2022           70   

                         Team1                 Team2  \
0             Rajasthan Royals        Gujarat Titans   
1  Royal Challengers Bangalore      Rajasthan Royals   
2  Royal Challengers Bangalore  Lucknow Super Giants   
3             Rajasthan Royals        Gujarat Titans   
4          Sunrisers Hyderabad          Punjab Kings   

                              Venue            TossWinner TossDecision  \
0  Narendra Modi Stadium, Ahmedabad      Rajasthan Royals          bat   
1  Narendra Modi Stadium, Ahmedabad      Rajasthan Royals        field   
2             Eden Gardens, Kolkata  Lucknow Super Giants        field   
3             Eden Gardens, Kolkata        Gujarat Titans        field   
4          Wankhede Stadium, Mumbai   Sunrisers Hyderabad          bat   

  SuperOver                  WinningTeam    WonBy  Margin method  \
0         N               Gujarat Titans  Wickets     7.0    NaN   
1         N             Rajasthan Royals  Wickets     7.0    NaN   
2         N  Royal Challengers Bangalore     Runs    14.0    NaN   
3         N               Gujarat Titans  Wickets     7.0    NaN   
4         N                 Punjab Kings  Wickets     5.0    NaN   

  Player_of_Match                                       Team1Players  \
0       HH Pandya  ['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...   
1      JC Buttler  ['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...   
2      RM Patidar  ['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...   
3       DA Miller  ['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...   
4   Harpreet Brar  ['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...   

                                        Team2Players        Umpire1  \
0  ['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...    CB Gaffaney   
1  ['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...    CB Gaffaney   
2  ['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...  J Madanagopal   
3  ['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...   BNJ Oxenford   
4  ['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...   AK Chaudhary   

         Umpire2  
0    Nitin Menon  
1    Nitin Menon  
2       MA Gough  
3      VK Sharma  
4  NA Patwardhan

In [7]:
data_ipl_2k23_player_team.head()

player Team 
0  YBK Jaiswal    RR
1   JC Buttler    RR
2    SV Samson    RR
3   D Padikkal    RR
4   SO Hetmyer    RR

## Match Dataset Preprocessing

### Data cleaning

In [9]:
data_match.columns

Index(['ID', 'City', 'Date', 'Season', 'MatchNumber', 'Team1', 'Team2',
       'Venue', 'TossWinner', 'TossDecision', 'SuperOver', 'WinningTeam',
       'WonBy', 'Margin', 'method', 'Player_of_Match', 'Team1Players',
       'Team2Players', 'Umpire1', 'Umpire2'],
      dtype='object')

In [10]:
data_match.isna().sum()

ID                   0
City                51
Date                 0
Season               0
MatchNumber          0
Team1                0
Team2                0
Venue                0
TossWinner           0
TossDecision         0
SuperOver            4
WinningTeam          4
WonBy                0
Margin              18
method             931
Player_of_Match      4
Team1Players         0
Team2Players         0
Umpire1              0
Umpire2              0
dtype: int64

In [37]:
winning_team=data_match[['ID','WinningTeam']]

ID                  WinningTeam
0    1312200               Gujarat Titans
1    1312199             Rajasthan Royals
2    1312198  Royal Challengers Bangalore
3    1312197               Gujarat Titans
4    1304116                 Punjab Kings
..       ...                          ...
945   335986        Kolkata Knight Riders
946   335985  Royal Challengers Bangalore
947   335984             Delhi Daredevils
948   335983          Chennai Super Kings
949   335982        Kolkata Knight Riders

[950 rows x 2 columns]